Importing Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_cs
import os
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPool1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time

# **Pre Processing**

Loading Embeddings

In [4]:
# we had to split our embeddings generator into two parts die to our hardware limitations
# Adobe dataset - content

# Load your embeddings from the two files
embedding_file_1 = "/kaggle/input/embeddings-adobe-bertweet/bertweet_full_content_head_embeddings.npy"
embedding_file_2 = "/kaggle/input/embeddings-adobe-bertweet/bertweet_full_content_tail_embeddings.npy"

embeddings_1 = np.load(embedding_file_1)
embeddings_2 = np.load(embedding_file_2)

# Concatenate the embeddings along the appropriate axis (axis=0 for vertical concatenation)
e1 = np.concatenate([embeddings_1, embeddings_2], axis=0)


In [5]:
# adobe dataset - likes

# Load your embeddings from the two files
embedding_file_1 = "/kaggle/input/embeddings-adobe-bertweet/bertweet_head_likes.npy"
embedding_file_2 = "/kaggle/input/embeddings-adobe-bertweet/bertweet_tail_likes.npy"

embeddings_1 = np.load(embedding_file_1)
embeddings_2 = np.load(embedding_file_2)

# Concatenate the embeddings along the appropriate axis (axis=0 for vertical concatenation)
t1 = np.concatenate([embeddings_1, embeddings_2], axis=0)

In [6]:
# hugging face dataset - content

# Load your embeddings from the two files
embedding_file_1 = "/kaggle/input/huggingf-dataset/full-content-huggingf-head.npy"
embedding_file_2 = "/kaggle/input/huggingf-dataset/full-content-huggingf-tail.npy"

embeddings_1 = np.load(embedding_file_1)
embeddings_2 = np.load(embedding_file_2)

# Concatenate the embeddings along the appropriate axis (axis=0 for vertical concatenation)
e2 = np.concatenate([embeddings_1, embeddings_2], axis=0)

In [7]:
# hugging face dataset - likes

embedding_file_1 = "/kaggle/input/huggingf-dataset/likes-hugginggf-head.npy"
embedding_file_2 = "/kaggle/input/huggingf-dataset/likes-hugginggf-tail.npy"

# Load embeddings with allow_pickle=True
embeddings_1 = np.load(embedding_file_1, allow_pickle=True)
embeddings_2 = np.load(embedding_file_2, allow_pickle=True)

# Concatenate the embeddings along the appropriate axis (axis=0 for vertical concatenation)
t2= np.concatenate((embeddings_1, embeddings_2), axis=0)


In [8]:
# 20 most followed people datasets

# content
e3=np.load("/kaggle/input/all-embeddings/full-content-add-ds.npy")

# likes
t3=np.load("/kaggle/input/all-embeddings/likes-add-ds.npy")


Creating Training dataset

In [12]:
# Combine embeddings and target values
X_train = np.concatenate([e1, e2, e3])
y_train = np.concatenate([t1, t2, t3])

# Shuffle the training data
shuffle_indices = np.random.permutation(len(X_train))
X_train = X_train[shuffle_indices]
y_train = y_train[shuffle_indices]


Splitting our Dataset into classes

In [14]:
# Create a DataFrame to combine embeddings and targets
train_data = pd.DataFrame({'embeddings': X_train.tolist(), 'targets': y_train})

# Filter out rows with non-integer values in the "targets" column
train_data = train_data[train_data['targets'].astype(str).str.isdigit()]

# Convert the "targets" column to integers
train_data['targets'] = train_data['targets'].astype(int)

# Define the class ranges
class_ranges = [(0, 100), (100, 1000), (1000, 5000), (5000, 10000), (10000, 100000), (100000, 1000000)]

# Initialize an empty dictionary to store DataFrames for each class
class_dfs = {}

# Process data in chunks
chunk_size = 10000  # Adjust the chunk size based on your available memory

# Create separate DataFrames for each class range
for idx, (lower, upper) in enumerate(class_ranges):
    class_name = f'class{idx + 1}'
    class_dfs[class_name] = pd.DataFrame()

    # Process data in chunks
    for start_idx in range(0, len(train_data), chunk_size):
        end_idx = start_idx + chunk_size

        # Filter data based on target range
        chunk_df = train_data[(train_data['targets'] >= lower) & (train_data['targets'] <= upper)].iloc[start_idx:end_idx]

        # Concatenate to the class DataFrame
        class_dfs[class_name] = pd.concat([class_dfs[class_name], chunk_df])

# Now class_dfs contains separate DataFrames for each class range
# You can access them using keys like class_dfs['class1'], class_dfs['class2'], etc.

# Extract embeddings and targets for each class
class_embeddings = {}
class_targets = {}

for class_name, class_df in class_dfs.items():
    class_embeddings[class_name] = np.array(class_df['embeddings'].to_list())
    class_targets[class_name] = class_df['targets'].values

# Now class_embeddings and class_targets contain separate arrays for each class range
# You can access them using keys like class_embeddings['class1'], class_targets['class2'], etc.


In [15]:
# Accessing classes
class_name = 'class1'
embeddings_class1 = class_embeddings[class_name]
targets_class1 = class_targets[class_name]

class_name = 'class2'
embeddings_class2 = class_embeddings[class_name]
targets_class2 = class_targets[class_name]

class_name = 'class3'
embeddings_class3 = class_embeddings[class_name]
targets_class3 = class_targets[class_name]

class_name = 'class4'
embeddings_class4 = class_embeddings[class_name]
targets_class4 = class_targets[class_name]

class_name = 'class5'
embeddings_class5 = class_embeddings[class_name]
targets_class5 = class_targets[class_name]

class_name = 'class6'
embeddings_class6 = class_embeddings[class_name]
targets_class6 = class_targets[class_name]


Training Regressors

# **LIKES : 0 - 100**

In [ ]:
X_train = embeddings_class1 
y_train = targets_class1


model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.1),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='linear')
])



# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['RootMeanSquaredError'])

# Define callbacks
callbacks = [
    ReduceLROnPlateau(patience=2),
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath='model-regression.h5', save_best_only=True)
]

from sklearn.model_selection import train_test_split
import time

# Assuming X_train and y_train are your training data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

print("Training started")
start = time.time()
history = model.fit(X_train, y_train, epochs=70, batch_size=32, validation_data=(X_val, y_val), callbacks=callbacks)
print("Training completed")
print("Time required: " + str(time.time() - start))

model.save("0_100_reg.h5")




# **LIKES: 100-1000**

In [ ]:
X_train = embeddings_class2 
y_train = targets_class2


model1 = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.1),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='linear')
])



# Compile the model
model1.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['RootMeanSquaredError'])

# Define callbacks
callbacks = [
    ReduceLROnPlateau(patience=2),
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath='model-regression.h5', save_best_only=True)
]

from sklearn.model_selection import train_test_split
import time

# Assuming X_train and y_train are your training data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

print("Training started")
start = time.time()
history = model1.fit(X_train, y_train, epochs=70, batch_size=32, validation_data=(X_val, y_val), callbacks=callbacks)
print("Training completed")
print("Time required: " + str(time.time() - start))

model1.save("100_1000_reg.h5")



# **LIKES: 1000-5000**

In [ ]:
X_train = embeddings_class3 
y_train = targets_class3


model2 = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.1),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='linear')
])



# Compile the model
model2.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['RootMeanSquaredError'])

# Define callbacks
callbacks = [
    ReduceLROnPlateau(patience=2),
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath='model-regression.h5', save_best_only=True)
]

from sklearn.model_selection import train_test_split
import time

# Assuming X_train and y_train are your training data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

print("Training started")
start = time.time()
history = model2.fit(X_train, y_train, epochs=70, batch_size=32, validation_data=(X_val, y_val), callbacks=callbacks)
print("Training completed")
print("Time required: " + str(time.time() - start))

model2.save("1000_5000_reg.h5")



# **LIKES: 5000-10000**

In [ ]:
X_train = embeddings_class4 
y_train = targets_class4


model3 = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.1),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='linear')
])



# Compile the model
model3.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['RootMeanSquaredError'])

# Define callbacks
callbacks = [
    ReduceLROnPlateau(patience=2),
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath='model-regression.h5', save_best_only=True)
]

from sklearn.model_selection import train_test_split
import time

# Assuming X_train and y_train are your training data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

print("Training started")
start = time.time()
history = model3.fit(X_train, y_train, epochs=70, batch_size=32, validation_data=(X_val, y_val), callbacks=callbacks)
print("Training completed")
print("Time required: " + str(time.time() - start))

model3.save("5000_10000_reg.h5")



# **LIKES: 10000-100000**

In [ ]:
X_train = embeddings_class5 
y_train = targets_class5

model4 = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.1),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='linear')
])



# Compile the model
model4.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['RootMeanSquaredError'])

# Define callbacks
callbacks = [
    ReduceLROnPlateau(patience=2),
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath='model-regression.h5', save_best_only=True)
]

from sklearn.model_selection import train_test_split
import time

# Assuming X_train and y_train are your training data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

print("Training started")
start = time.time()
history = model4.fit(X_train, y_train, epochs=70, batch_size=32, validation_data=(X_val, y_val), callbacks=callbacks)
print("Training completed")
print("Time required: " + str(time.time() - start))
model4.save("10000_100000_reg.h5")



# **LIKES: 100000-1000000**

In [ ]:
X_train = embeddings_class6
y_train = targets_class6


model5 = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.1),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='linear')
])



# Compile the model
model5.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['RootMeanSquaredError'])

# Define callbacks
callbacks = [
    ReduceLROnPlateau(patience=2),
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath='model-regression.h5', save_best_only=True)
]

from sklearn.model_selection import train_test_split
import time

# Assuming X_train and y_train are your training data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

print("Training started")
start = time.time()
history = model5.fit(X_train, y_train, epochs=70, batch_size=32, validation_data=(X_val, y_val), callbacks=callbacks)
print("Training completed")
print("Time required: " + str(time.time() - start))

model5.save("100000_1000000_reg.h5")


